# Entregable 1
## Inteligencia Artificial

En este entregable se realizará la aplicación de Análisis Exploratorio de Datos (EDA), y los clasificadores K vecinos más cercanos (k-NN) y árboles de decisión.

Para la solución deben escoger algún conjunto de datos para **clasificación** de kaggle (https://www.kaggle.com/datasets) o UCI (https://archive.ics.uci.edu/). 

Con los datos seleccionados hacer lo siguiente:

1. Realizar un Análisis Exploratorio de Datos del conjunto de datos seleccionado. En este paso indicar qué se hace y para qué, es decir, realizar un análisis consciente de lo que se está realizando y con qué fin.

2. Realizar una división del conjunto de datos en entrenamiento (train) y prueba (test), la proporción de división es libre.

3. Con el conjunto de datos depurado realizar el entrenamiento de un modelo de k-NN. Encontrar el valor de k óptimo para la solución del problema con el conjunto de prueba.

4. Finalmente, realizar el entrenamiento de un clasificador de árbol de decisión. Realizar la evaluación de este clasificador para diferentes hiperparámetros, buscando encontrar los mejores para el conjunto de prueba.

In [1]:
#celda para importar, pueden agregar las que requieran

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Solución a punto 1.

In [2]:
df = pd.read_csv('chip_dataset.csv')
print(df.shape)
df.head()

(4854, 14)


,Unnamed: 0,Product,Type,Release Date,Process Size (nm),TDP (W),Die Size (mm^2),Transistors (million),Freq (MHz),Foundry,Vendor,FP16 GFLOPS,FP32 GFLOPS,FP64 GFLOPS
0,0,AMD Athlon 64 3500+,CPU,2007-02-20,65.0,45.0,77.0,122.0,2200.0,Unknown,AMD,NaN,NaN,NaN
1,1,AMD Athlon 200GE,CPU,2018-09-06,14.0,35.0,192.0,4800.0,3200.0,Unknown,AMD,NaN,NaN,NaN
2,2,Intel Core i5-1145G7,CPU,2020-09-02,10.0,28.0,NaN,NaN,2600.0,Intel,Intel,NaN,NaN,NaN
3,3,Intel Xeon E5-2603 v2,CPU,2013-09-01,22.0,80.0,160.0,1400.0,1800.0,Intel,Intel,NaN,NaN,NaN
4,4,AMD Phenom II X4 980 BE,CPU,2011-05-03,45.0,125.0,258.0,758.0,3700.0,Unknown,AMD,NaN,NaN,NaN


In [5]:
df.any().isna()

Unnamed: 0               False
Product                  False
Type                     False
Release Date             False
Process Size (nm)        False
TDP (W)                  False
Die Size (mm^2)          False
Transistors (million)    False
Freq (MHz)               False
Foundry                  False
Vendor                   False
FP16 GFLOPS              False
FP32 GFLOPS              False
FP64 GFLOPS              False
dtype: bool

In [7]:
df = df.iloc[: , :-3]

In [8]:
df

,Unnamed: 0,Product,Type,Release Date,Process Size (nm),TDP (W),Die Size (mm^2),Transistors (million),Freq (MHz),Foundry,Vendor
0,0,AMD Athlon 64 3500+,CPU,2007-02-20,65.0,45.0,77.0,122.0,2200.0,Unknown,AMD
1,1,AMD Athlon 200GE,CPU,2018-09-06,14.0,35.0,192.0,4800.0,3200.0,Unknown,AMD
2,2,Intel Core i5-1145G7,CPU,2020-09-02,10.0,28.0,NaN,NaN,2600.0,Intel,Intel
3,3,Intel Xeon E5-2603 v2,CPU,2013-09-01,22.0,80.0,160.0,1400.0,1800.0,Intel,Intel
4,4,AMD Phenom II X4 980 BE,CPU,2011-05-03,45.0,125.0,258.0,758.0,3700.0,Unknown,AMD
...,...,...,...,...,...,...,...,...,...,...,...
4849,4849,NVIDIA Quadro 3000M,GPU,2011-02-22,40.0,75.0,332.0,1950.0,450.0,TSMC,NVIDIA
4850,4850,Intel GMA 950,GPU,2005-06-01,90.0,7.0,NaN,NaN,250.0,Intel,Intel
4851,4851,NVIDIA GeForce GT 320M,GPU,2010-03-03,40.0,23.0,100.0,486.0,500.0,TSMC,NVIDIA
4852,4852,NVIDIA GeForce FX 5200,GPU,2003-03-06,150.0,NaN,65.0,29.0,250.0,TSMC,NVIDIA


In [9]:
num_nan = df.isna().sum()
# print(num_nan.to_string())
print(num_nan[num_nan != 0])

Process Size (nm)          9
TDP (W)                  626
Die Size (mm^2)          715
Transistors (million)    711
dtype: int64


In [10]:
df[num_nan[num_nan != 0].index].isna()

,Process Size (nm),TDP (W),Die Size (mm^2),Transistors (million)
0,False,False,False,False
1,False,False,False,False
2,False,False,True,True
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
4849,False,False,False,False
4850,False,False,True,True
4851,False,False,False,False
4852,False,True,False,False


In [11]:
df = df.dropna(subset=num_nan[num_nan != 0].index, how='any')
df

,Unnamed: 0,Product,Type,Release Date,Process Size (nm),TDP (W),Die Size (mm^2),Transistors (million),Freq (MHz),Foundry,Vendor
0,0,AMD Athlon 64 3500+,CPU,2007-02-20,65.0,45.0,77.0,122.0,2200.0,Unknown,AMD
1,1,AMD Athlon 200GE,CPU,2018-09-06,14.0,35.0,192.0,4800.0,3200.0,Unknown,AMD
3,3,Intel Xeon E5-2603 v2,CPU,2013-09-01,22.0,80.0,160.0,1400.0,1800.0,Intel,Intel
4,4,AMD Phenom II X4 980 BE,CPU,2011-05-03,45.0,125.0,258.0,758.0,3700.0,Unknown,AMD
5,5,Intel Xeon E5-2470 v2,CPU,2013-09-01,22.0,95.0,160.0,1400.0,2400.0,Intel,Intel
...,...,...,...,...,...,...,...,...,...,...,...
4844,4844,ATI FirePro V7800,GPU,2010-04-26,40.0,150.0,334.0,2154.0,700.0,TSMC,ATI
4845,4845,NVIDIA Tegra 3 GPU,GPU,2011-11-09,40.0,20.0,80.0,10.0,416.0,TSMC,NVIDIA
4846,4846,NVIDIA Playstation 3 GPU 28nm,GPU,2013-06-03,28.0,21.0,68.0,302.0,550.0,Sony,NVIDIA
4849,4849,NVIDIA Quadro 3000M,GPU,2011-02-22,40.0,75.0,332.0,1950.0,450.0,TSMC,NVIDIA


In [13]:
df.isnull().sum().max()

0

## Solución a punto 2.

## Solución a punto 3.

## Solución a punto 4.